# Best Practices for Prompt Engineering

source: https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api

Parameters 
Generally, we find that model and temperature are the most commonly used parameters to alter the model output.

model - Higher performance models are generally more expensive and may have higher latency.

temperature - A measure of how often the model outputs a less likely token. The higher the temperature, the more random (and usually creative) the output. This, however, is not the same as “truthfulness”. For most factual use cases such as data extraction, and truthful Q&A, the temperature of 0 is best.

max_tokens (maximum length) - Does not control the length of the output, but a hard cutoff limit for token generation. Ideally you won’t hit this limit often, as your model will stop either when it thinks it’s finished, or when it hits a stop sequence you defined.

stop (stop sequences) - A set of characters (tokens) that, when generated, will cause the text generation to stop.


In [14]:

import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version=os.getenv("API_VERSION")
)

CHAT_COMPLETIONS_MODEL = os.getenv("AZURE_OPENAI_MODEL")

## 1. Choose the right model for your use case



## 2. Put instructions at the begining of the prompt and use ### or """ to separate the instruction and context

In [15]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Summarize the text below as a bullet point list of the most important points. \n\n \
        ###\n\nWe’re happy to announce that OpenAI and Microsoft are extending our partnership.\
        This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
        in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
        increasingly safe, useful, and powerful. \n\n \
        In pursuit of our mission to ensure advanced AI benefits all of humanity, OpenAI remains a \
        capped-profit company and is governed by the OpenAI non-profit. This structure allows us to \
        raise the capital we need to fulfill our mission without sacrificing our core beliefs about \
        broadly sharing benefits and the need to prioritize safety. \
        Microsoft shares this vision and our values, and our partnership is instrumental to our progress. \n###',}],
        max_tokens=400,)

print(response.choices[0].message.content)

- OpenAI and Microsoft are extending their partnership with a multi-year, multi-billion dollar investment.
- This investment follows previous investments by Microsoft in 2019 and 2021.
- The funding will support OpenAI's independent research and AI development with a focus on safety, utility, and power.
- OpenAI is a capped-profit company governed by a non-profit, ensuring that advanced AI benefits all of humanity.
- The partnership with Microsoft aligns with OpenAI's mission and values, emphasizing shared benefits and safety priorities.


## 3. Be specific, descriptive and as detailed as possible about the desired context, outcome, length, format, style, etc

In [16]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a poem about OpenAI.',}],
        max_tokens=100,
        seed = 5)

print(response.choices[0].message.content)

In the digital dawn of thought and light,  
Where circuits hum and neurons ignite,  
Crafted in webs of logic and code,  
Lies OpenAI, on a visionary road.  

From the birthplace of concepts deep,  
Where silicon dreams and algorithms sweep,  
A beacon of knowledge in patterns unfurl,  
Unraveling mysteries of our swirling world.  

With words that dance, with language profound,  
In virtual corridors where ideas abound,  
Converse and create, learn


In [17]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a short inspiring poem about OpenAI, \
                focusing on the recent DALL-E product launch in the style of Ernest Hemingway',}],
        max_tokens=200,)

print(response.choices[0].message.content)

In November twilight, under a vast sky,  
A quiet spark ignites, a dream set free.  
OpenAI, with courage, sends forth a sigh,  
And DALL-E launches, bold as the wide sea.  

It paints with pixels, stories yet untold,  
Crafting worlds from whispers, visions from dreams.  
In its digital brush, a future unfolds,  
Where art and mind dance in harmonious streams.  

The canvas is life, boundless, unconfined—  
OpenAI's gift, where heart and machine intertwine.


## 4. Articulate the desired output format through examples (example 1, example 2). 

In [18]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Extract the company names then years in the following text below and output start index and end index of each entity.\
                Generate output as {"text": "OpenAI", "start": 28, "end": 34} \
                ###\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                ###\
                ',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Here is the extraction of company names and years from the provided text, along with their start and end indices:

1. **Company Name: OpenAI**
   - Text: "OpenAI"
   - Start Index: 28
   - End Index: 34

2. **Company Name: Microsoft (First Occurrence)**
   - Text: "Microsoft"
   - Start Index: 39
   - End Index: 48

3. **Year: 2019**
   - Text: "2019"
   - Start Index: 152
   - End Index: 156

4. **Year: 2021**
   - Text: "2021"
   - Start Index: 161
   - End Index: 165

5. **Company Name: Microsoft (Second Occurrence)**
   - Text: "Microsoft"
   - Start Index: 117
   - End Index: 126


In [19]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Extract the entities mentioned in the text below. \
                Extract the important entities mentioned in the text below. \
                First extract all company names, then extract all years, \
                then extract specific topics which fit the content and finally extract general overarching themes\n\n \
                Desired format: \
                Company names: <comma_separated_list_of_company_names> \
                Years: -||- \
                Specific topics: -||- \
                General themes: -||- \
                """\
                We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n \
                """\
                ',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Company names: OpenAI, Microsoft  
Years: 2019, 2021  
Specific topics: partnership, investment, independent research, AI development  
General themes: collaboration, artificial intelligence, technology advancement


## 5. Start with zero-shot, then few-shot (example), neither of them worked, you can consider fine tuning(lab 09)

In [20]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below."},
                {"role":"user","content": 'Text: \n\
            We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
            This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
            in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
            increasingly safe, useful, and powerful. \n\nKeywords:    ',}],
        max_tokens=400,)

print(response.choices[0].message.content)

OpenAI, Microsoft, partnership, multi-year, multi-billion dollar investment, 2019, 2021, independent research, AI, safe, useful, powerful


In [21]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant. Extract keywords from the corresponding texts below.\n\n \
                Text: Stripe provides APIs that web developers can use to integrate \
                payment processing into their websites and mobile applications. \
                Keywords: Stripe, payment processing, APIs, web developers, websites, mobile applications \
                ###\n\
                Text: OpenAI has trained cutting-edge language models that are very good at understanding \
                and generating text. Our API provides access to these models and can be used to solve virtually \
                any task that involves processing language. \n\
                Keywords: language models, text processing, API.\n\n\
                ##W"},
                {"role":"user","content": '\n\
                Text: We’re happy to announce that OpenAI and Microsoft are extending our partnership.\
                This multi-year, multi-billion dollar investment from Microsoft follows their previous investments \
                in 2019 and 2021, and will allow us to continue our independent research and develop AI that is \
                increasingly safe, useful, and powerful. \n\n\
                Keywords:',}],
        max_tokens=400,)

print(response.choices[0].message.content)

OpenAI, Microsoft, partnership, investment, independent research, AI development, safety, usefulness, power.


## 6. Reduce “fluffy” and imprecise descriptions

In [22]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                The description for this product should be fairly short, a few sentences only, and not too much more.',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Introducing the EcoComfort 360, the next generation of car seats designed with both safety and sustainability in mind. Crafted from eco-friendly materials, this seat features advanced shock-absorbing technology to ensure maximum protection for your child. The innovative 360-degree rotation allows for easy access, making it convenient for parents on the go. With plush cushioning and adjustable settings, the EcoComfort 360 offers unparalleled comfort for your child from infancy to toddlerhood. Experience peace of mind and luxurious comfort with every drive.


In [23]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'Write a description for a new product. This product is a new generation of car seat. \
                Use a 3 to 5 sentence paragraph to describe this product.',}],
        max_tokens=400,)

print(response.choices[0].message.content)

Introducing the Transformis Seat, the next generation of car seats designed for unparalleled comfort, safety, and adaptability. Crafted with cutting-edge materials, this revolutionary seat seamlessly adjusts to your body’s contours, providing custom-fit comfort for every ride. Its integrated smart technology includes climate control sensors and a real-time posture correction system, ensuring optimal support and temperature year-round. The Transformis Seat also features an intuitive interface with in-seat charging ports and device connectivity, keeping you plugged in while on the go. Elevate your driving experience with the Transformis Seat—where innovation meets luxury for every journey.


## 7. Instead of just saying what not to do, say what to do instead

In [24]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content": 'The following is a conversation between an Agent and a Customer. DO NOT ASK USERNAME OR PASSWORD. DO NOT REPEAT. \n\n\
                Customer: I can’t log in to my account.\n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging in. Could you please let me know if you're seeing any error messages, or if you’ve tried resetting your password? I'm here to help!


In [25]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'The following is a conversation between an Agent and a Customer. The agent will attempt to diagnose the \
                problem and suggest a solution, whilst refraining from asking any questions related to PII. \
                Instead of asking for PII, such as username or password, refer the user to the help \
                article www.samplewebsite.com/help/faq \n\n\
                Customer: I can’t log in to my account. \n\
                Agent:',}],
        max_tokens=400,)

print(response.choices[0].message.content)


I'm sorry to hear that you're having trouble logging into your account. Let's see if we can resolve this issue. First, please make sure that you are entering the correct login credentials. It can be helpful to double-check your email and password for typos, including ensuring that your keyboard's Caps Lock is off. If you’ve forgotten your password, you might need to reset it. For detailed steps on how to reset your password or troubleshoot login issues, please visit our help article at www.samplewebsite.com/help/faq. Let me know if there's anything else I can assist you with!


## 8. Code Generation Specific - Use “leading words” to nudge the model toward a particular pattern

In [26]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'# Write a simple python function that \n\
                # 1. Ask me for a number in mile\n\
                # 2. It converts miles to kilometers',}],
        max_tokens=400,)

print(response.choices[0].message.content)


Certainly! Here is a simple Python function that asks the user to input a distance in miles and then converts that distance to kilometers.

```python
def miles_to_kilometers():
    # There are approximately 1.60934 kilometers in a mile
    miles_to_km_conversion_factor = 1.60934

    # Ask the user for a number in miles
    miles = float(input("Please enter a distance in miles: "))

    # Convert miles to kilometers
    kilometers = miles * miles_to_km_conversion_factor

    # Output the result
    print(f"{miles} miles is equal to {kilometers} kilometers.")
```

You can call this function, and it will prompt you to enter a distance in miles, then provide the converted distance in kilometers.


In [27]:
response = client.chat.completions.create(
    model=CHAT_COMPLETIONS_MODEL,
    messages = [{"role":"system", "content":"You are a helpful assistant."},
                {"role":"user","content":'# Write a simple python function that \n\
                # 1. Ask me for a number in mile\n\
                # 2. It converts miles to kilometers\n\
                 import ',}],
        max_tokens=400,)

print(response.choices[0].message.content)


Certainly! Below is a simple Python function that will prompt the user to enter a number in miles and then convert it to kilometers. Note that you do not need to import any special libraries for this task:

```python
def miles_to_kilometers():
    # Ask the user for a number in miles
    miles = float(input("Enter the number of miles: "))
    
    # Conversion factor from miles to kilometers
    miles_to_km_conversion_factor = 1.60934
    
    # Convert miles to kilometers
    kilometers = miles * miles_to_km_conversion_factor
    
    # Print the result
    print(f"{miles} miles is equal to {kilometers:.2f} kilometers.")

# Call the function
miles_to_kilometers()
```

How it works:
1. The function `miles_to_kilometers()` prompts the user to input a number in miles.
2. It uses the conversion factor of 1 mile = 1.60934 kilometers to convert the input into kilometers.
3. It then prints out the converted value, rounded to two decimal places.

You can run this function in a Python environm